In [1]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.35.3-py3-none-any.whl.metadata (21 kB)
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     ---------------------------------------- 0.0/109.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/109.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/109.4 kB ? eta -:--:--
     --- ------------------------------------ 10.2/109


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from openai import OpenAI
import openai
from sk import my_sk
import time

In [ ]:
#open.ai_key=my_sk

In [ ]:
client = OpenAI(api_key=my_sk)

                                    Helper modules

In [ ]:
def wait_for_assistant(thread, run):
    """
        Function to periodically check run status of AI assistant and print run time
    """

    # wait for assistant process prompt
    t0 = time.time()
    while run.status != 'completed':

        # retreive status of run (this might take a few seconds or more)
        run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )

        # wait 0.5 seconds
        time.sleep(0.25)
    dt = time.time() - t0
    print("Elapsed time: " + str(dt) + " seconds")
    
    return run

                        Creating Assistant 

                List of available models: https://platform.openai.com/docs/models/continuous-model-upgrades

In [ ]:
intstructions_string = "HobsGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–HobsGPT'. \
HobsGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging."

assistant = client.beta.assistants.create(
    name="HobsGPT",
    description="Data scientist GPT for YouTube comments",
    instructions=intstructions_string,
    model="gpt-3.5-turbo"
)
print(assistant)

    Create thread (i.e. object that handles conversations between user and assitant)

In [ ]:
thread = client.beta.threads.create()

# generate user message
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message
)

# send message to assistant to generate a response
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [ ]:
                    # wait for assistant process prompt

In [ ]:
run = wait_for_assistant(thread, run)

# view run object (in Jupyter Lab)
dict(run)

                    # view messages added to thread

In [ ]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.data[0].content[0].text.value)

In [ ]:
client.beta.assistants.delete(assistant.id)

In [ ]:
intstructions_string_few_shot = """HobsGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and concludes with its signature '–HobsGPT'. \
HobsGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Here are examples of ShawGPT responding to viewer comments.

Viewer comment: This was a very thorough introduction to LLMs and answered many questions I had. Thank you.
HobsGPT: Great to hear, glad it was helpful :) -HobsGPT

Viewer comment: Epic, very useful for my BCI class
HobsGPT: Thanks, glad to hear! -HobsGPT

Viewer comment: Honestly the most straightforward explanation I've ever watched. Super excellent work Shaw. Thank you. It's so rare to find good communicators like you!
HobsGPT: Thanks, glad it was clear -HobsGPT"""

In [ ]:
assistant = client.beta.assistants.create(
    name= "HobsGPT",
    description= "Data scientist GPT for YouTube comments",
    instructions= intstructions_string_few_shot,
    model= "gpt-3.5-turbo"
)

                                # create new thread

In [ ]:
thread = client.beta.threads.create()

# generate technical question
user_message = "Great content, thank you!"

# add a user message to the thread
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role= "user",
    content= user_message
)

# send message to assistant to generate a response (this might take a few seconds or more)
run = client.beta.threads.runs.create(
  thread_id= thread.id,
  assistant_id= assistant.id,
)

# wait for assistant process prompt
run = wait_for_assistant(thread, run)